# Exercice 3   

**Question 1**: Modelisation du problème de plus court chemin comme problème du flot de coût minimum

$G=(V,E)$, source: $s \in V$, puit: $t \in V$

$a(u,v)$: cout pour passer de u à v, $f(u,v)=1$ si $(u,v) \in $ {plus court chemin} 

Programme linéaire:
\begin{align*}
\min&\sum_{(u,v)\in E} a(u,v)f(u,v)&\\
s.c.& \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,u) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
    & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}\\
    &f(u,v)\in (0,1) \quad \forall (u,v) \in E&
\end{align*}

In [45]:
couts = {
    ('a','b'): 4,
    ('a','c'): 5,
    ('b','c'): 2,
    ('b','d'): 1,
    ('b','e'): 2,
    ('b','f'): 7,
    ('c','d'): 5,
    ('c','e'): 2,
    ('d','f'): 3,
    ('e','f'): 5
}

noeuds = set()
for arc in couts:
    noeuds.update(arc)
noeuds = list(noeuds)

source = min(noeuds)
puits = max(noeuds)
source, puits

('a', 'f')

In [46]:
from gurobipy import *

In [47]:
len(couts.keys())

10

In [61]:
model = Model()
nombre_arcs = len(couts)
f = model.addVars(couts.keys(), vtype = GRB.BINARY, name='f')
model.setObjective(quicksum([f[i,j] * cout for (i,j),cout in couts.items()]), GRB.MINIMIZE)

# contrainte de conservation du flux
for v in noeuds:
    if v == source or v == puits:
        continue
    flux_entrant = quicksum([f[i,v] for i,j in couts if j == v])
    flux_sortant = quicksum([f[v,j] for i,j in couts if i == v])
    model.addConstr(flux_entrant == flux_sortant)

#for arc, cout in couts.items():
    #model.addConstr(f[arc] <= cout )    # contrainte de capacité
    #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "elementary OS 7.1 Horus")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5 rows, 10 columns and 17 nonzeros
Model fingerprint: 0x410a0458
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 17 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 12.0000000
Found heuristic solution: objective 11.0000000
Found heuristic solution: objective 8.0000000

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [62]:
objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]
objets_selectiones

[('a', 'b'), ('b', 'd'), ('d', 'f')]